# Touristic Tour Recommendation Application
This notebook outlines the steps involved in creating an algorithm that generates a one-week itinerary for tourists in Algeria. The itinerary is optimized based on user preferences, proximity, and travel costs. Various search techniques, including **Uninformed Search Algorithms**, **A*** and **Hill Climbing**, are employed to create the optimal itinerary.


## Data Collection & Research
We gathered - clean - data about **+100 Algerian tourist attractions**, including the following attributes:
- **Attraction Name**
- **Type of Attraction** (museum, nature, beach, etc.)
- **City**
- **Cost** (entry fee)
- **Rating** (user rating)
- **GPS Coordinates** (latitude, longitude)
- **Description** (short description)


In [1]:
import json
from collections import Counter

DATA_PATH = "../Data/attractions.json"

with open(DATA_PATH, "r", encoding="utf-8") as f:
    attractions_data = json.load(f)
    

if not isinstance(attractions_data, list):
    raise ValueError("The JSON file does not contain a list of attractions.")

print("Number of attractions:", len(attractions_data))

# Count attractions per city
city_counts = Counter(attraction.get("city", "Unknown") for attraction in attractions_data)

# Count attractions per category
category_counts = Counter(attraction.get("category", "Unknown") for attraction in attractions_data)

print("\nNumber of attractions per city:")
for city, count in city_counts.items():
    print(f"{city}: {count}")

print("\nNumber of attractions per category:")
for category, count in category_counts.items():
    print(f"{category}: {count}")


Number of attractions: 153

Number of attractions per city:
Algiers: 18
Tipaza: 2
Blida: 1
Médéa: 3
Oran: 16
Tlemcen: 9
Batna: 1
Ghardaïa: 2
Béjaïa: 3
Constantine: 6
Djanet: 1
Sétif: 13
Annaba: 4
Guelma: 3
El Tarf: 6
Tamanrasset: 3
Béchar: 1
Bouira: 1
El Bayadh: 1
Khenchela: 1
Biskra: 4
Timimoun: 3
El Oued: 1
M'Sila: 1
Tizi Ouzou: 10
Beni Abbes: 1
Skikda: 4
Souk Ahras: 2
Tébessa: 3
Oum El-Bouaghi: 5
Jijel: 20
Aïn Témouchent: 2
Boumerdès: 2

Number of attractions per category:
Garden: 3
Museum: 10
Cultural: 18
Historical: 25
Religious: 9
Amusement Park: 5
Port: 2
Shopping Mall: 4
Nature: 47
Resort: 1
Beach: 22
Lake: 1
Island: 1
Wildlife Park: 1
Coastal Town: 1
Recreational: 1
Wellness: 2


In [3]:
import math
import json
import re
from collections import deque

########################################################
# Utility Functions
########################################################

def parse_duration(duration_str):
    """
    Convert a 'visit_duration' string (e.g. '2 hours', '1-2 hours', '0.5 hour')
    into a single float. We'll just capture the first numeric found.
    """
    if not duration_str:
        return 1.0  # default if missing

    match = re.search(r"(\d+(\.\d+)?)", duration_str)
    if match:
        return float(match.group(1))
    return 1.0

def parse_entrance_cost(cost_str):
    """
    Convert a 'cost' field (e.g. '300 DZD', 'Free', 'Variable') into a float.
    Example:
        '300 DZD' -> 300.0
        'Free' or None -> 0.0
        'Variable' -> 0.0
    """
    if not cost_str or cost_str.lower().startswith("free"):
        return 0.0
    match = re.search(r"\d+", cost_str)
    return float(match.group()) if match else 0.0

def haversine_distance(gps1, gps2):
    """
    Approx. distance in kilometers between two lat/lon points using Haversine.
    gps1, gps2 are [lat, lon].
    """
    R = 6371  # Earth radius in km
    lat1, lon1 = map(math.radians, gps1)
    lat2, lon2 = map(math.radians, gps2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def travel_cost_km(gps_a, gps_b):
    """
    Suppose we set a cost rate of 10 DZD per km traveled.
    Adjust as needed for your application.
    """
    dist = haversine_distance(gps_a, gps_b)
    return dist * 10.0

def matches_interest(attraction, user_interests):
    """
    Check if the attraction's category is among the user's interests.
    e.g. user_interests = ["Museum", "Garden"], etc.
    """
    return attraction["category"] in user_interests

def to_immutable_day_plans(day_plans):
    """
    Convert a list-of-lists like:
      [
        ["Attraction1", "Attraction2"],   # day 1
        ["Attraction3"],                  # day 2
        ...
      ]
    into a tuple-of-tuples so it's hashable and can go into a set/dict.
    """
    return tuple(tuple(day_list) for day_list in day_plans)

def from_immutable_day_plans(immutable_plans):
    """
    Convert the tuple-of-tuples back into a list-of-lists for modification.
    """
    return [list(day_tuple) for day_tuple in immutable_plans]

########################################################
# BFS: Multi-Stop (One or More Attractions per Day)
########################################################

def bfs_itinerary_with_multiple_stops(
    user_start_city,
    daily_hours,
    total_budget,
    user_interests,
    attractions_data
):
    """
    A BFS that allows multiple visits in a single day if time/budget permit,
    for a total of 7 days max.
    
    day_plans is a list (length 7) where each element is a list of visited attractions that day.
    
    Returns a list-of-lists representing a 7-day plan, or None if no plan is found.
    """

    # Build a dict for quick lookups by attraction name
    name_map = {a["name"]: a for a in attractions_data}

    # Find all possible "start attractions" in the user's start city that match interests
    start_candidates = []
    for attr in attractions_data:
        if attr["city"].lower() == user_start_city.lower():
            if matches_interest(attr, user_interests):
                start_candidates.append(attr)

    # BFS queue: each element is a tuple:
    # (
    #   day (1..7),
    #   hours_left_in_that_day,
    #   current_attraction_name,
    #   budget_left,
    #   visited_set_of_names,
    #   day_plans_as_immutable
    # )
    queue = deque()
    visited_states = set()

    # Initialize BFS with each possible first attraction
    for start_attr in start_candidates:
        visit_time = parse_duration(start_attr.get("visit_duration", "1 hour"))
        entrance_fee = parse_entrance_cost(start_attr.get("cost", "Free"))

        # Optionally, define travel cost from "hotel/home" to first attraction. We'll assume 0.
        initial_travel_cost = 0.0
        total_cost = initial_travel_cost + entrance_fee

        if total_cost > total_budget:
            continue  # can't afford even the first place

        if visit_time > daily_hours:
            continue  # can't fit in day 1 time

        # Build an empty 7-day plan; day=1 means index=0
        new_plans_list = [[] for _ in range(7)]
        new_plans_list[0].append(start_attr["name"])  # day-1 index is 0

        # Remaining daily hours after visiting this place
        leftover_hours = daily_hours - visit_time
        leftover_budget = total_budget - total_cost

        # Make an immutable version for BFS storage
        immutable_plans = to_immutable_day_plans(new_plans_list)

        init_state = (
            1,                     # day
            leftover_hours,
            start_attr["name"],    # current location
            leftover_budget,
            frozenset([start_attr["name"]]),
            immutable_plans
        )

        queue.append(init_state)
        visited_states.add(init_state)

    # BFS Loop
    while queue:
        day, hours_left, current_name, budget_left, visited, plans_immutable = queue.popleft()

        # If day == 7 and we still have hours_left >= 0, we've assigned 7 days
        # (the BFS expansions will only add more visits on day 7 if they fit)
        # If we don't want to force 7 full days, we can check if day=7 and can't add more visits => done
        if day == 7 and hours_left >= 0:
            # Convert from immutable structure to list-of-lists to return
            final_plan = from_immutable_day_plans(plans_immutable)
            return final_plan

        current_attr = name_map[current_name]

        # We'll attempt to add a next attraction (either same day or next day):
        for next_attr in attractions_data:
            next_name = next_attr["name"]
            if next_name in visited:
                continue  # skip if already visited

            # Must match user's interest
            if not matches_interest(next_attr, user_interests):
                continue

            # Calculate costs:
            t_cost = travel_cost_km(current_attr["gps"], next_attr["gps"])  # travel cost
            e_cost = parse_entrance_cost(next_attr.get("cost", "Free"))     # entrance fee
            v_time = parse_duration(next_attr.get("visit_duration", "1 hour"))

            total_next_cost = t_cost + e_cost
            if total_next_cost > budget_left:
                continue  # not enough money left

            # Suppose we also consider driving time:
            # e.g., 1 hour driving per 50 km => distance / 50
            dist_km = haversine_distance(current_attr["gps"], next_attr["gps"])
            drive_hours = dist_km / 50.0
            total_time_needed = v_time + drive_hours

            ########################################
            # 1) Try visiting on the SAME day
            ########################################
            if total_time_needed <= hours_left:
                new_day = day
                new_hours_left = hours_left - total_time_needed
                new_budget_left = budget_left - total_next_cost
                new_visited = visited | {next_name}

                # convert day_plans back to a list for modification
                day_plans_list = from_immutable_day_plans(plans_immutable)
                day_plans_list[new_day - 1].append(next_name)

                # now convert it to an immutable structure
                new_plans_immutable = to_immutable_day_plans(day_plans_list)

                new_state = (
                    new_day,
                    new_hours_left,
                    next_name,
                    new_budget_left,
                    frozenset(new_visited),
                    new_plans_immutable
                )

                if new_state not in visited_states:
                    visited_states.add(new_state)
                    queue.append(new_state)

            ########################################
            # 2) Try moving to the NEXT day (day+1)
            ########################################
            if day < 7:
                next_day = day + 1
                # On a new day, we have daily_hours again, 
                # but must subtract travel_time + v_time
                if total_time_needed <= daily_hours:
                    new_hours_left = daily_hours - total_time_needed
                    new_budget_left = budget_left - total_next_cost
                    new_visited = visited | {next_name}

                    # copy current day plans
                    day_plans_list = from_immutable_day_plans(plans_immutable)
                    # put next_name on day index (next_day - 1)
                    day_plans_list[next_day - 1].append(next_name)

                    new_plans_immutable = to_immutable_day_plans(day_plans_list)

                    new_state = (
                        next_day,
                        new_hours_left,
                        next_name,
                        new_budget_left,
                        frozenset(new_visited),
                        new_plans_immutable
                    )
                    if new_state not in visited_states:
                        visited_states.add(new_state)
                        queue.append(new_state)

    # If we exhaust all possibilities without a 7-day solution, return None
    return None


########################################################
# Example Usage
########################################################

if __name__ == "__main__":
    DATA_PATH = "../Data/attractions.json"
    with open(DATA_PATH, "r", encoding="utf-8") as f:
        attractions_data = json.load(f)

    user_start_city = "Algiers"  # e.g. user says "I'm in Algiers"
    daily_hours = 8.0            # max hours per day
    total_budget = 15000.0       # DZD
    user_interests = ["Museum", "Historical", "Garden", "Nature"]

    day_plans = bfs_itinerary_with_multiple_stops(
        user_start_city,
        daily_hours,
        total_budget,
        user_interests,
        attractions_data
    )

    if day_plans is None:
        print("No feasible 7-day itinerary found under these constraints.")
    else:
        print("Found a 7-day plan!\n")
        # Let's display each day's attractions
        for i, day_list in enumerate(day_plans, start=1):
            print(f"Day {i}: {day_list}")


MemoryError: 

## Distance and Proximity Calculation
To calculate the optimal itinerary, we need to compute the distances between attractions. We will use the **Haversine Formula** to calculate the distance between two locations based on their GPS coordinates.


In [2]:
import math

# Haversine formula to calculate distance between two points (in km)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# e.g.
distance = haversine(36.7452, 3.0750, 36.5894, 2.4477)
distance


58.570295060336306

## Helper Functions

In [ ]:
def parse_duration(duration_str):
    """
    Convert a duration string like "1-2 hours", "2-4 hours", or "3 hours"
    to a numeric estimate (e.g., hours as a float).
    For simplicity, we'll pick the lower bound or an average.
    """
    # Examples: "1-2 hours" -> we might take 1.5 as an average
    #           "3 hours"   -> 3.0
    #           "Variable..." -> let's pick a default
    import re
    
    # for a pattern like "X-Y hours"
    match_range = re.match(r"(\d+)-(\d+)\s*hours", duration_str.lower())
    if match_range:
        low = float(match_range.group(1))
        high = float(match_range.group(2))
        return (low + high) / 2.0
    
    # for a pattern like "X hours"
    match_single = re.match(r"(\d+)\s*hours", duration_str.lower())
    if match_single:
        return float(match_single.group(1))
    
    # If "Variable", "Unknown", or any other text
    return 2.0  # DEFAULT ! ?

def parse_cost(cost_str):
    """
    Convert the cost field (like "Free", "400 DZD", "Unknown", or "Variable")
    into a numeric value (DZD). For simplicity:
      - "Free" -> 0
      - "400 DZD" -> 400
      - else -> let's guess 200 as a placeholder
    """
    cost_str_lower = cost_str.lower()
    if "free" in cost_str_lower:
        return 0
    import re
    match = re.search(r"(\d+)", cost_str)
    if match:
        # Return first integer found
        return int(match.group(1))
    # Otherwise "variable", "unknown", etc.
    return 200  # DEFAULT ! ?


## User Preferences Simulation 
The application will take inputs (from the website interface) such as:
- **Starting location**: Algiers
- **Preferred attractions**: Museums, Historical Sites
- **Budget**: 1500 DZD
- **Hotel rating**: 3 stars


## Problem Formulation
**1. Define the Problem Components** 

We model the itinerary planning as a **search problem** and a **CSP**.  
- **States**: Partial itineraries (sequence of attractions).  
- **Actions**: Adding a new attraction to the itinerary.  
- **Goal**: A 7-day itinerary that maximizes user satisfaction and minimizes cost/time.  
- **Constraints**: No repeated attractions, budget limits, and proximity.

**2. State Representation**

In this formulation, the **state** represents a partial itinerary for the traveler. It includes the list of visited attractions, remaining attractions, total cost so far, and total time spent so far.

**Node Class**

A `Node` class is used to represent each state in the search space. It holds information about the current state, parent node, action taken, cumulative costs (`g`), and the evaluation function (`f`).

In [20]:
import copy
from math import radians, sin, cos, sqrt, atan2

class Node:
    def __init__(self, state, parent=None, action=None, g=0, f=0):
        """
        Initialize a search tree node.

        Input Parameters:
            - state: The state represented by this node (e.g., a city name).
                     Example: "Algiers"
            - parent: The parent Node that generated this node. Default is None.
            - action: The action taken to reach this node from the parent. Often the same as the state.
                      Example: "Oran"
            - g: The cumulative cost (actual cost) from the start node to this node. Default is 0.
            - f: The evaluation function value for the node (e.g., for UCS, A*). Default is 0.

        Output:
            - A Node instance with attributes: state, parent, action, g, f, and depth.
        """
        self.state = state
        self.parent = parent
        self.action = action
        self.g = g  # Cumulative cost from start to this node
        self.f = f  # Evaluation cost (g + heuristic if applicable)
        # Calculate the depth of the node
        if parent is None:
            self.depth = 0
        else:
            self.depth = parent.depth + 1

    def __hash__(self):
        """
        Compute a hash value for the node.

        Input Parameters:
            - None (uses the node's state)

        Output:
            - An integer hash value.
        """
        if isinstance(self.state, list):
            state_tuple = tuple([tuple(row) for row in self.state])
            return hash(state_tuple)
        return hash(self.state)

    def __eq__(self, other):
        """
        Check equality with another Node based on the state.

        Input Parameters:
            - other: Another Node instance.

        Output:
            - True if the states are equal, False otherwise.
        """
        return isinstance(other, Node) and self.state == other.state

    def __gt__(self, other):
        """
        Compare this node with another node based on the evaluation function (f).

        Input Parameters:
            - other: Another Node instance.

        Output:
            - True if this node's f is greater than the other's f, else False.
        """
        return isinstance(other, Node) and self.f > other.f

**3. Actions**

Actions are the valid attractions that can be added to the itinerary based on constraints.

**4. Goal Test**

Check if the itinerary is complete (7 days) and meets all constraints.

In [ ]:
class TouristicTourProblem:
    def __init__(self, attractions, preferences, start_location, budget, max_time):
        """
        Initialize the touristic tour problem.

        Input Parameters:
            - attractions: List of available attractions with details (name, category, cost, gps, etc.)
            - preferences: A dictionary of user preferences (e.g., type of attractions preferred).
            - start_location: The starting city for the traveler.
            - budget: The total budget for the trip (in local currency).
            - max_time: The maximum time available for the trip (in hours).
        """
        self.attractions = attractions
        self.preferences = preferences
        self.start_location = start_location
        self.budget = budget
        self.max_time = max_time
        self.visited_attractions = []
        self.remaining_attractions = attractions

    def is_goal(self, state):
        """Check if the tour meets the constraints for budget, time, and satisfaction."""
        # Check if total time spent on the selected attractions is within the maximum allowed
        if state["total_time"] > self.max_time:
            return False

        # Check if total cost is within the maximum allowed budget
        if state["total_cost"] > self.max_budget:
            return False

        # Check if user preferences are satisfied
        satisfaction_score = self.calculate_satisfaction(state)
        if satisfaction_score < self.user_preferences["min_satisfaction"]:
            return False

        # CAN add more conditions as needed, e.g., ensuring the user visited a certain category of attractions
        return True
    
    def calculate_satisfaction(self, state):
        """Calculate a satisfaction score based on user preferences."""
        satisfaction = 0
        for attraction in state["visited"]:
            # Score attractions based on categories that the user prefers
            if self.preferences.get("nature") and attraction["category"] == "Nature":
                satisfaction += attraction["rating"]
            if self.preferences.get("cultural") and attraction["category"] == "Cultural":
                satisfaction += attraction["rating"]
            if self.preferences.get("historical") and attraction["category"] == "Historical":
                satisfaction += attraction["rating"]
            # CAN add more categories and refine the logic as needed
        return satisfaction

    def get_valid_actions(self, state):
        """
        Get valid actions (next attractions to visit) from the current state.
        
        Input:
            - state: The current state (includes visited and remaining attractions).
        
        Output:
            - List of possible actions (attractions to visit next).
        """
        return state["remaining"]

    def transition(self, state, attraction):
        """
        Transition function: Move from one state to another after visiting an attraction.
        
        Input:
            - state: Current state (visited and remaining attractions).
            - attraction: The attraction to visit next.
        
        Output:
            - New state after the transition.
        """
        new_visited = state["visited"] + [attraction]
        new_remaining = [a for a in state["remaining"] if a != attraction]
        
        # Calculate the cost and time for the selected attraction
        cost = self.calculate_cost(attraction)
        time = self.calculate_time(attraction)
        
        # Create the new state
        new_state = {
            "visited": new_visited,
            "remaining": new_remaining,
            "total_cost": state["total_cost"] + cost,
            "total_time": state["total_time"] + time,
            "preferences": state["preferences"]
        }
        return new_state

    def calculate_cost(self, attraction):
        """
        Calculate the cost to visit an attraction.
        
        Input:
            - attraction: The attraction to visit.
        
        Output:
            - The cost to visit the attraction (in local currency).
        """
        # Here we assume the cost is given as part of the attraction data
        return int(attraction["cost"].split()[0])  # Parsing the DZD cost

    def calculate_time(self, attraction):
        """
        Calculate the time to visit an attraction.
        
        Input:
            - attraction: The attraction to visit.
        
        Output:
            - The time to visit the attraction (in hours).
        """
        return float(attraction["visit_duration"].split()[0])  # Assuming duration is given in hours

    def heuristic(self, state):
        """
        Heuristic function to estimate the cost/time to visit the remaining attractions.

        Input:
            - state: The current state.

        Output:
            - Estimated cost to complete the tour (remaining cost and time).
        """
        remaining_cost = sum([self.calculate_cost(a) for a in state["remaining"]])
        remaining_time = sum([self.calculate_time(a) for a in state["remaining"]])
        return remaining_cost + remaining_time

def is_goal_state(state, max_time):
    """
    Check if the current state is a goal state.
    
    Input:
        - state: The current state.
        - max_time: The maximum time available for the trip.
    
    Output:
        - True if all attractions are visited, and the total time and cost are within the limits.
    """
    total_time = state["total_time"]
    total_cost = state["total_cost"]
    return len(state["remaining"]) == 0 and total_time <= max_time and total_cost <= state["preferences"]["budget"]


## Search Algorithm Implementations

In this project, we use a combination of **uninformed search algorithms** (BFS, DFS) and **informed search algorithms** (A\* Search, Hill Climbing) to generate the optimal itinerary based on cost, proximity, and user preferences.

Each search algorithm has its strengths and weaknesses. In this section, we will implement and compare their performance in solving the itinerary optimization problem.


### BFS
Explores all paths at the current depth before moving on to the next level, ensuring the shortest path is found if the graph is unweighted.


In [8]:
from collections import deque

def bfs(): # parameters needed !
    """
    Perform a BFS to find the shortest path from start to goal.
    
    Args:
    - start: Starting location (current location).
    - goal: Desired goal (final destination).
    - attractions: List of all attractions (places to visit).
    - distance_function: Function to calculate the distance between locations.
    - maybe more, depends on the one implementing the algorithm
    
    Returns:
    - List of attractions in the optimal path.
    """
    pass

# maybe other functions here

### DFS
Explores each path as deep as possible before backtracking, often used when exploring solutions in a depth-first manner.

In [9]:
def dfs(): # parameters needed !
    """
    Perform a DFS to find a path from start to goal.
    
    Args:
    - start: Starting location.
    - goal: Desired goal.
    - attractions: List of all attractions.
    - distance_function: Function to calculate distance.
    - maybe more, depends on the one implementing the algorithm
    
    Returns:
    - List of attractions in the path, or None if no path is found.
    """
    pass

# maybe other functions here

### A\*
Uses a heuristic to optimize the search process by combining the current cost with the estimated cost to reach the goal.

In [10]:
import heapq

def a_star_search(): # parameters needed !
    """
    Perform A* search to find the optimal path from start to goal.
    
    Args:
    - start: Starting point.
    - goal: Goal point.
    - attractions: List of all attractions.
    - distance_function: Function to calculate distance between locations.
    - heuristic_function: Heuristic function to estimate the cost from a point to the goal.
    - maybe more, depends on the one implementing the algorithm
    
    Returns:
    - List of attractions in the optimal path.
    """
    pass


### Hill-Climbing
A simple greedy approach that evaluates only the immediate next state and chooses the best option available without considering future paths.

In [11]:
def hill_climb_search(): # parameters needed !
    """
    Perform Hill Climbing to find the optimal path based on local evaluations.
    
    Args:
    - start: Starting location.
    - attractions: List of all attractions.
    - distance_function: Function to calculate distance.
    - heuristic_function: Function to evaluate the "goodness" of a path.
    - maybe more, depends on the one implementing the algorithm
    
    Returns:
    - List of attractions in the path, or None if no solution is found.
    """
    pass


## CSP Approach
We model the itinerary planning as a **Constraint Satisfaction Problem (CSP)**. Each attraction represents a variable, and the domain of each variable is the set of attractions that can be visited. Constraints include:
  1. **One destination per day**: Each day gets exactly one destination.
  2. **Proximity and time**: Destinations must be scheduled in a way that minimizes travel time and adheres to time constraints.
  3. **Cost**: The total cost of the trip should be within the user’s budget.
  4. **Preferences**: Ensure that the user’s preferences for types of attractions are respected.


In [12]:
# Define constraints for the CSP
def check_constraints(itinerary, user_preferences):
    pass


## Comparative Evaluation of Algorithms 

We will compare the performance of the following search algorithms:

1. **BFS**: Guarantees the shortest path but may be slow for large datasets.
2. **DFS**: Fast but not guaranteed to find the optimal solution.
3. **A\***: Combines optimality with efficiency when the heuristic is accurate.
4. **Hill Climbing**: Greedy and fast but may get stuck in local optima.
5. **CSP Approach**: Will compare the CSP solution with the search-based methods.

We will evaluate the following metrics:
- **Execution time**: How long each algorithm takes to generate the itinerary.
- **Path optimality**: How close the solution is to the optimal itinerary in terms of cost, proximity, and user satisfaction.
- **Complexity**: The computational complexity of each approach.


In [5]:
def evaluate_solution(itinerary, total_cost, time_taken):
    # Compare the total cost, time efficiency, and other factors
    pass  # To be implemented


## Visualizations

We will include several visualizations to help understand the results:
- **Route Map**: Visualize the itinerary on a map.
- **Cost Breakdown**: Graph showing the total cost per destination.
- **Satisfaction**: Bar chart comparing user satisfaction for different search strategies.


In [6]:
import matplotlib.pyplot as plt

# Example plot for the cost breakdown
def plot_cost_breakdown(itinerary, attractions):
    costs = [attractions[a]['cost'] for a in itinerary]
    plt.bar(range(len(itinerary)), costs)
    plt.xlabel('Attraction')
    plt.ylabel('Cost')
    plt.title('Cost Breakdown')
    plt.show()


## Demo

In this section, we will showcase the working prototype of the **Touristic Tour Recommendation Application**. This demo will cover:
- The **interactive input** from the user (e.g., preferences, current location, etc.).
- Displaying the **optimized itinerary** generated by the selected search algorithm.
- Visualizations of the **travel route**, **cost breakdown**, and **satisfaction level**.

In [13]:
pass

## Conclusion

In this notebook, we explored different search algorithms (A\*, Hill Climbing, BFS, DFS) and a CSP approach to generate optimal itineraries for travelers in Algeria. 
We found that **.....** performed well in terms of solution quality, but it was more computationally expensive compared to **.....**. 
Future work could involve integrating real-time weather data and optimizing routes based on current traffic conditions.
